<a href="https://colab.research.google.com/github/malongfei1993/chatglm3-6b-qlora-finetuning/blob/main/qlora_chatglm3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# QLoRA加载chatglm-6b，peft进行微调


*   根据构建的数据集进行微调
*   把微调后的peft模型部分进行保存





In [ ]:
import torch

print(torch.__config__.show(), torch.cuda.get_device_properties(0))

PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v3.1.1 (Git Hash 64f6bcbcbab628e96f33a62c3e975f8535a7bde4)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX512
  - CUDA Runtime 12.1
  - NVCC architecture flags: -gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=sm_60;-gencode;arch=compute_70,code=sm_70;-gencode;arch=compute_75,code=sm_75;-gencode;arch=compute_80,code=sm_80;-gencode;arch=compute_86,code=sm_86;-gencode;arch=compute_90,code=sm_90
  - CuDNN 8.9.2
  - Magma 2.6.1
  - Build settings: BLAS_INFO=mkl, BUILD_TYPE=Release, CUDA_VERSION=12.1, CUDNN_VERSION=8.9.2, CXX_COMPILER=/opt/rh/devtoolset-9/root/usr/bin/c++, CXX_FLAGS= -D_GLIBCXX_USE_CXX11_ABI=0 -fabi-version=11 -fvisibility-inlines-hidden -DUSE_PTHREADPOOL -DNDEBUG -DUSE_KINET

In [ ]:
# 定义全局变量和参数
model_name_or_path = 'THUDM/chatglm3-6b'
train_data_path = 'drive/MyDrive/DSData/ds_data.csv'    # 训练数据路径(批量生成数据集）
eval_data_path = None
seed = 8
#max_input_length = 2048
max_input_length = 21000
max_output_length = 700
lora_rank = 16
lora_alpha = 32
lora_dropout = 0.05
prompt_text = ''

In [ ]:
# !pip uninstall transformers

In [ ]:
# !pip install transformers==4.34.0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip show transformers

Name: transformers
Version: 4.34.0
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: 


In [ ]:
!pip show datasets

In [ ]:

!pip install sentencepiece
!pip install bitsandbytes
!pip install accelerate
!pip install peft
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.1/330.1 kB 16.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.5 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.17.3
    Uninstalling huggingface-hub-0.17.3:
      Successfully uninstalled huggingface-hub-0.17.3


In [ ]:
from datasets import load_dataset

dataset = load_dataset("csv", data_files=train_data_path)
print(dataset)

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['customer', 'parameter', 'content', 'summary', 'json_data'],
        num_rows: 35
    })
})


In [ ]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(dataset["train"], num_examples=5)

,customer,parameter,content,summary,json_data
0,None,TLCM reflectance,"\n这是数据：\n {\n ""ID"": ""ID"",\n ""General characteristics"": ""Optical Characteristics"",\n ""Information"": ""Information"",\n ""Value"": ""Value"",\n ""Response"": NaN\n },\n {\n ""ID"": 4.1,\n ""General characteristics"": ""Light reflection (AR)"",\n ""Information"": ""d/8\u00b0 measurement as per IDMS \u00a711.2"",\n ""Value"": ""SCE 0.3% , SCI <0.5%"",\n ""Response"": NaN\n },\n {\n ""ID"": 4.2,\n ""General characteristics"": ""Surface reflection"",\n ""Information"": ""Reflection(Y)\u3000*Does not include internal reflection such as LCD."",\n ""Value"": "" Haze 6.5%+/- 2% "",\n ""Response"": NaN\n },\n----------\n请回复如下格式：\n[{\n""参数"":"""",\n""需求规格"":"""",\n""""供应商反馈""""：""""\n}]\n你的任务：\n提取TLCM reflectance的参数规格需求信息和供应商反馈信息。\n下面是你的回复(使用英文)：\n","\n{\n""参数"": ""TLCM reflectance"",\n""需求规格"": ""SCE 0.3% , SCI <0.5%"",\n""供应商反馈"": """"\n}\n","{\n ""ID"": ""ID"",\n ""General characteristics"": ""Optical Characteristics"",\n ""Information"": ""Information"",\n ""Value"": ""Value"",\n ""Response"": NaN\n },\n {\n ""ID"": 4.1,\n ""General characteristics"": ""Light reflection (AR)"",\n ""Information"": ""d/8\u00b0 measurement as per IDMS \u00a711.2"",\n ""Value"": ""SCE 0.3% , SCI <0.5%"",\n ""Response"": NaN\n },\n {\n ""ID"": 4.2,\n ""General characteristics"": ""Surface reflection"",\n ""Information"": ""Reflection(Y)\u3000*Does not include internal reflection such as LCD."",\n ""Value"": "" Haze 6.5%+/- 2% "",\n ""Response"": NaN\n },"
1,None,black luminance homogeneity,"\n这是数据：\n {\n ""No."": 39,\n ""General characteristics"": ""LCF"",\n ""Unnamed: 3"": NaN,\n ""requirement"": ""n.a."",\n ""Quote option"": ""<="",\n ""comment Supplier"": NaN\n },\n {\n ""No."": 40,\n ""General characteristics"": ""Luminance homogeneity 9-point VESA"",\n ""Unnamed: 3"": NaN,\n ""requirement"": ""<80% white; <70% black image"",\n ""Quote option"": ""<="",\n ""comment Supplier"": ""min 80%@9 points, white\nmin 60%@9 points, black""\n },\n {\n ""No."": 41,\n ""General characteristics"": ""Response time"",\n ""Unnamed: 3"": ""Ta=-30\u00b0C (grey/grey)"",\n ""requirement"": ""<200ms"",\n ""Quote option"": ""<="",\n ""comment Supplier"": ""Max 400ms, TG2G 42Gray step, Tij=Tij90%-Tij10%""\n },\n----------\n请回复如下格式：\n[{\n""参数"":"""",\n""需求规格"":"""",\n""""供应商反馈""""：""""\n}]\n你的任务：\n提取black luminance homogeneity的参数规格需求信息和供应商反馈信息。\n下面是你的回复(使用英文)：\n","\n{\n""参数"": ""black luminance homogeneity"",\n""需求规格"": ""<70% black image "",\n""供应商反馈"": ""min 60%@9 points, black""\n}\n",None
2,None,white luminance homogeneity,"\n这是数据：\n {\n ""No."": 39,\n ""General characteristics"": ""LCF"",\n ""Unnamed: 3"": NaN,\n ""requirement"": ""n.a."",\n ""Quote option"": ""<="",\n ""comment Supplier"": NaN\n },\n {\n ""No."": 40,\n ""General characteristics"": ""Luminance homogeneity 9-point VESA"",\n ""Unnamed: 3"": NaN,\n ""requirement"": ""<80% white; <70% black image"",\n ""Quote option"": ""<="",\n ""comment Supplier"": ""min 80%@9 points, white\nmin 60%@9 points, black""\n },\n {\n ""No."": 41,\n ""General characteristics"": ""Response time"",\n ""Unnamed: 3"": ""Ta=-30\u00b0C (grey/grey)"",\n ""requirement"": ""<200ms"",\n ""Quote option"": ""<="",\n ""comment Supplier"": ""Max 400ms, TG2G 42Gray step, Tij=Tij90%-Tij10%""\n },\n----------\n请回复如下格式：\n[{\n""参数"":"""",\n""需求规格"":"""",\n""""供应商反馈""""：""""\n}]\n你的任务：\n提取white luminance homogeneity的参数规格需求信息和供应商反馈信息。\n下面是你的回复(使用英文)：\n","\n{\n""参数"": ""white luminance homogeneity"",\n""需求规格"": ""<80% white; "",\n""供应商反馈"": ""min 80%@9 points, white""\n}\n","{\n ""No."": 39,\n ""General characteristics"": ""LCF"",\n ""Unnamed: 3"": NaN,\n ""requirement"": ""n.a."",\n ""Quote option"": ""<="",\n ""comment Supplier"": NaN\n },\n {\n ""No."": 40,\n ""General characteristics"": ""Luminance homogeneity 9-point VESA"",\n ""Unnamed: 3"": NaN,\n ""requirement"": ""<80% white; <70% black image"",\n ""Quote option"": ""<="",\n ""comment Supplier"": ""min 80%@9 points

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/518 [00:00<?, ?B/s]

tokenization_chatglm.py:   0%|          | 0.00/13.0k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/THUDM/chatglm3-6b:
- tokenization_chatglm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer.model:   0%|          | 0.00/1.02M [00:00<?, ?B/s]

In [ ]:
# import pandas as pd
# df_read=pd.read_csv('drive/MyDrive/DSData/Forvia_outputs/Forvia_Aug_without_output_summary.csv')
# len(tokenizer.tokenize(df_read['summary'][9]))

In [ ]:
# tokenize_func 函数
def tokenize_func(example, tokenizer, ignore_label_id=-100):
    """
    对单个数据样本进行tokenize处理。

    参数:
    example (dict): 包含'content'和'summary'键的字典，代表训练数据的一个样本。
    tokenizer (transformers.PreTrainedTokenizer): 用于tokenize文本的tokenizer。
    ignore_label_id (int, optional): 在label中用于填充的忽略ID，默认为-100。

    返回:
    dict: 包含'tokenized_input_ids'和'labels'的字典，用于模型训练。
    """

    # 构建问题文本
    question = prompt_text + example['content']
    if example.get('input', None) and example['input'].strip():
        question += f'\n{example["input"]}'

    # 构建答案文本
    answer = example['summary']

    # 对问题和答案文本进行tokenize处理
    q_ids = tokenizer.encode(text=question, add_special_tokens=False)
    a_ids = tokenizer.encode(text=answer, add_special_tokens=False)

    # 如果tokenize后的长度超过最大长度限制，则进行截断
    if len(q_ids) > max_input_length - 2:  # 保留空间给gmask和bos标记
        q_ids = q_ids[:max_input_length - 2]
    if len(a_ids) > max_output_length - 1:  # 保留空间给eos标记
        a_ids = a_ids[:max_output_length - 1]

    # 构建模型的输入格式
    input_ids = tokenizer.build_inputs_with_special_tokens(q_ids, a_ids)
    question_length = len(q_ids) + 2  # 加上gmask和bos标记

    # 构建标签，对于问题部分的输入使用ignore_label_id进行填充
    labels = [ignore_label_id] * question_length + input_ids[question_length:]

    return {'input_ids': input_ids, 'labels': labels}


In [ ]:
#len(tokenized_dataset['labels'][0])

In [ ]:
#len(tokenized_dataset['input_ids'][0])

In [ ]:
#len(tokenizer.tokenize(dataset['train']['summary'][0]))+len(tokenizer.tokenize(dataset['train']['content'][0]))

In [ ]:
column_names = dataset['train'].column_names
tokenized_dataset = dataset['train'].map(
    lambda example: tokenize_func(example, tokenizer),
    batched=False,
    remove_columns=column_names
)

Using bos_token, but it is not set yet.
Using sep_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using mask_token, but it is not set yet.


Map:   0%|          | 0/35 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset = tokenized_dataset.shuffle(seed=seed)
tokenized_dataset = tokenized_dataset.flatten_indices()

Flattening the indices:   0%|          | 0/35 [00:00<?, ? examples/s]

In [ ]:
import torch
from typing import List, Dict, Optional

# DataCollatorForChatGLM 类
class DataCollatorForChatGLM:

    def __init__(self, pad_token_id: int, max_length: int = 2048, ignore_label_id: int = -100):

        self.pad_token_id = pad_token_id
        self.ignore_label_id = ignore_label_id
        self.max_length = max_length

    def __call__(self, batch_data: List[Dict[str, List]]) -> Dict[str, torch.Tensor]:

        # 计算批量中每个样本的长度
        len_list = [len(d['input_ids']) for d in batch_data]
        batch_max_len = max(len_list)  # 找到最长的样本长度

        input_ids, labels = [], []
        for len_of_d, d in sorted(zip(len_list, batch_data), key=lambda x: -x[0]):
            pad_len = batch_max_len - len_of_d  # 计算需要填充的长度
            # 添加填充，并确保数据长度不超过最大长度限制
            ids = d['input_ids'] + [self.pad_token_id] * pad_len
            label = d['labels'] + [self.ignore_label_id] * pad_len
            if batch_max_len > self.max_length:
                ids = ids[:self.max_length]
                label = label[:self.max_length]
            input_ids.append(torch.LongTensor(ids))
            labels.append(torch.LongTensor(label))

        # 将处理后的数据堆叠成一个tensor
        input_ids = torch.stack(input_ids)
        labels = torch.stack(labels)

        return {'input_ids': input_ids, 'labels': labels}


In [ ]:
# 准备数据整理器
data_collator = DataCollatorForChatGLM(pad_token_id=tokenizer.pad_token_id)

## 加载模型

In [ ]:
from transformers import AutoModel, BitsAndBytesConfig

_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])
# 加载量化后模型
model = AutoModel.from_pretrained(model_name_or_path,
                                  quantization_config=q_config,
                                  device_map='auto',
                                  trust_remote_code=True)

model.supports_gradient_checkpointing = True
model.gradient_checkpointing_enable()
model.enable_input_require_grads()

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

config.json:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

configuration_chatglm.py:   0%|          | 0.00/2.33k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/THUDM/chatglm3-6b:
- configuration_chatglm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_chatglm.py:   0%|          | 0.00/55.7k [00:00<?, ?B/s]

quantization.py:   0%|          | 0.00/14.7k [00:00<?, ?B/s]

...

model-00004-of-00007.safetensors:   0%|          | 0.00/1.82G [00:00<?, ?B/s]

model-00005-of-00007.safetensors:   0%|          | 0.00/1.97G [00:00<?, ?B/s]

model-00006-of-00007.safetensors:   0%|          | 0.00/1.93G [00:00<?, ?B/s]

model-00007-of-00007.safetensors:   0%|          | 0.00/1.05G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
from peft import TaskType, LoraConfig, get_peft_model, prepare_model_for_kbit_training
from peft.utils import TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING

kbit_model = prepare_model_for_kbit_training(model)
target_modules = TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING['chatglm']

In [ ]:
!pip show transformers

Name: transformers
Version: 4.34.0
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: peft


In [ ]:
lora_config = LoraConfig(
    target_modules=target_modules,
    r=lora_rank,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    bias='none',
    inference_mode=False,
    task_type=TaskType.CAUSAL_LM
)

In [ ]:
qlora_model = get_peft_model(kbit_model, lora_config)
qlora_model.print_trainable_parameters()

trainable params: 3,899,392 || all params: 6,247,483,392 || trainable%: 0.06241540401681151


### QLoRA 微调模型

In [ ]:
train_epochs = 4
output_dir = f"models/{model_name_or_path}-epoch{train_epochs}"

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=10,
    #per_device_train_batch_size=8,
    gradient_accumulation_steps=1,
    learning_rate=1e-3,
    num_train_epochs=train_epochs,
    lr_scheduler_type="linear",
    warmup_ratio=0.1,
    logging_steps=1,
    save_strategy="steps",
    save_steps=5,
    optim="adamw_torch",
    fp16=True,
)


In [ ]:
trainer = Trainer(
        model=qlora_model,
        args=training_args,
        train_dataset=tokenized_dataset,
        data_collator=data_collator
    )

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,1.150100
2,1.083400
3,0.618100
4,0.439200
5,0.430600
6,0.235900
7,0.265100
8,0.084300
9,0.082600
10,0.246100


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

TrainOutput(global_step=16, training_loss=0.3331401627510786, metrics={'train_runtime': 228.3396, 'train_samples_per_second': 0.613, 'train_steps_per_second': 0.07, 'total_flos': 2649886519787520.0, 'train_loss': 0.3331401627510786, 'epoch': 4.0})

In [ ]:
trainer.model.save_pretrained(output_dir)

In [ ]:
!cp -r models drive/MyDrive/fineTuning0201